# Reconstructing for lignocellulose degradation and uptake

### Common functions

In [1]:
import pandas as pd
from reframed import Metabolite, GPRAssociation, Gene, Protein, ReactionType, CBReaction, load_cbmodel, save_cbmodel
from reframed.io.sbml import parse_gpr_rule
import json

In [2]:
def add_metabolites(model, df):

    for index, row in df.iterrows():
        met = Metabolite(met_id=row['Identifier'], name=row['Name'], compartment="C_"+row['Compartment'])
        met.metadata={'FORMULA':row['Formula'], 'CHARGE':str(row['Charge'])}                  
        model.add_metabolite(met)

        if met.compartment=="C_e":
            rxn_id = "R_EX_" + row['Identifier']
            name = "Exchange of " + row['Name']
            rxn = CBReaction(reaction_id=rxn_id, name=name, reversible=True, stoichiometry={met.id: -1.0} , reaction_type=ReactionType.EXCHANGE)
            model.add_reaction(rxn)

In [3]:
gene_id_map = pd.read_csv('../input/uniprot-proteome_UP000001349.tab', sep='\t', usecols=[3,4], names=['name', 'refseq'], header=0)
gene_id_map['name'] =  gene_id_map['name'].str.split()
gene_id_map = gene_id_map.explode('name').dropna()
gene_id_map['refseq'] = gene_id_map['refseq'].apply(lambda x: f'G_{x[:-3]}_1')
gene_id_map = dict(gene_id_map.values)

In [4]:
def gene_str_to_GPR(gene_string):
    gpr = parse_gpr_rule(gene_string)
    
    for protein in gpr.proteins:
        protein.genes = [gene_id_map.get(x, 'G_' + x) for x in protein.genes]
                
    return gpr

In [5]:
def add_reactions(model, df):

    for index, row in df.iterrows():
        
        reaction_id = row['Identifier']
        name = row['Enzyme']
        reversible = False
        stoichiometry = json.loads(row['Stoichiometry'])
        reaction_type = ReactionType.ENZYMATIC
        
        if row['Transport']==1:
            reaction_type=ReactionType.TRANSPORT 
        
        rxn = CBReaction(reaction_id=reaction_id, name=name, reversible=reversible, stoichiometry=stoichiometry, reaction_type=reaction_type)
        model.add_reaction(rxn)
        
        gpr = gene_str_to_GPR(row['Gene'])
        model.set_gpr_association(rxn.id, gpr)

### Load model

In [6]:
model = load_cbmodel('../models/RcH10_draft.xml')

### Cellulose pathway

In [7]:
metabolites = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx', sheet_name="Mets.  Cat. of cellodextrins ", usecols="A:E").dropna()
reactions = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx', sheet_name="Rxns.  Cat. of cellodextrins ", usecols="A:F").dropna()

In [8]:
add_metabolites(model, metabolites)
add_reactions(model, reactions)

### Xyloglucan pathway

In [9]:
metabolites = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx', sheet_name="Mets. Cat. of xyloglu. oligosac", usecols="A:E").dropna()
reactions = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx',sheet_name="Rxns. Cat. of xyloglu. oligosac", usecols="A:F").dropna()

In [10]:
add_metabolites(model, metabolites)
add_reactions(model, reactions)

### Arabinoxylan pathway

In [11]:
metabolites = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx', sheet_name="Mets. arabinoxylan", usecols="A:E").dropna()
reactions = pd.read_excel('../input//Polysaccharide degrading pathways.xlsx', sheet_name="Rxns. Cat. of ara.xyl oligosac", usecols="A:F").dropna()
add_metabolites(model, metabolites)
add_reactions(model, reactions)

### Save model

In [12]:
save_cbmodel(model, '../models/RcH10_v1.xml')